In [1]:
import os
import pickle
import sklearn
import pandas as pd
import numpy as np
# os.chdir('/Users/dethf/NYU/Spring 2019/Chunara Internship/nyu-twipsy/')

In [2]:
clf_tob = pickle.load(open('./tobacco classifiers/classifier1_tobacco.p', 'rb'))

Processing data.

In [3]:
with open('./data/SF_tweets_2017_01.json', 'r') as f:
    data = f.readlines()

# remove the trailing "\n" from each line
data = map(lambda x: x.rstrip(), data)

# each element of 'data' is an individual JSON object. I want to convert it into an *array* of JSON objects 
# which, in and of itself, is one large JSON object basically... add square brackets to the beginning
# and end, and have all the individual business JSON objects separated by a comma.
data_json_str = "[" + ','.join(data) + "]"

df = pd.read_json(data_json_str)

In [4]:
df.columns

Index(['contributors', 'coordinates', 'created_at', 'display_text_range',
       'entities', 'extended_entities', 'extended_tweet', 'favorite_count',
       'favorited', 'filter_level', 'geo', 'id', 'id_str',
       'in_reply_to_screen_name', 'in_reply_to_status_id',
       'in_reply_to_status_id_str', 'in_reply_to_user_id',
       'in_reply_to_user_id_str', 'is_quote_status', 'lang', 'place',
       'possibly_sensitive', 'quoted_status', 'quoted_status_id',
       'quoted_status_id_str', 'retweet_count', 'retweeted', 'source', 'text',
       'timestamp_ms', 'truncated', 'user'],
      dtype='object')

In [5]:
df.head(3)

,contributors,coordinates,created_at,display_text_range,entities,extended_entities,extended_tweet,favorite_count,favorited,filter_level,...,quoted_status,quoted_status_id,quoted_status_id_str,retweet_count,retweeted,source,text,timestamp_ms,truncated,user
0,NaN,"{'type': 'Point', 'coordinates': [-122.416533,...",2017-01-04 21:51:34,"[0, 111]","{'hashtags': [], 'urls': [{'url': 'https://t.c...","{'media': [{'id': 816764030060621824, 'id_str'...",NaN,0,False,low,...,NaN,NaN,NaN,0,False,"<a href=""http://spotreporters.com"" rel=""nofoll...",Opened Street or Sidewalk Cleaning request via...,2017-01-04 21:51:34.849,False,"{'id': 1589692776, 'id_str': '1589692776', 'na..."
1,NaN,"{'type': 'Point', 'coordinates': [-122.433851,...",2017-01-12 21:34:11,NaN,"{'hashtags': [], 'urls': [{'url': 'https://t.c...",NaN,NaN,0,False,low,...,NaN,NaN,NaN,0,False,"<a href=""http://spotreporters.com"" rel=""nofoll...",Opened Street or Sidewalk Cleaning request via...,2017-01-12 21:34:11.692,False,"{'id': 1589692776, 'id_str': '1589692776', 'na..."
2,NaN,"{'type': 'Point', 'coordinates': [-122.514, 37...",2017-01-17 23:23:49,NaN,"{'hashtags': [], 'urls': [{'url': 'https://t.c...",NaN,NaN,0,False,low,...,NaN,NaN,NaN,0,False,"<a href=""http://instagram.com"" rel=""nofollow"">...","Just posted a photo @ Cliff House, San Francis...",2017-01-17 23:23:49.139,False,"{'id': 883339848, 'id_str': '883339848', 'name..."


In [6]:
print(len(df))

87764


In [7]:
df.text[:10]

0    Opened Street or Sidewalk Cleaning request via...
1    Opened Street or Sidewalk Cleaning request via...
2    Just posted a photo @ Cliff House, San Francis...
3    Search for bank robbery suspect shuts down Hig...
4    Opened Graffiti request via android at 4134 Mi...
5    Opened Street or Sidewalk Cleaning request via...
6    Closed Graffiti request at Intersection of Bra...
7    William Rhodes's work call for Beauty @ Root D...
8     #DarkestPossibleTimeline https://t.co/AglO5D5hNx
9    Opened Street or Sidewalk Cleaning request via...
Name: text, dtype: object

In [112]:
df_text = df[['text']] # instead of df.text, which will only return Series object.

In [9]:
df_text.head(20)

,text
0,Opened Street or Sidewalk Cleaning request via...
1,Opened Street or Sidewalk Cleaning request via...
2,"Just posted a photo @ Cliff House, San Francis..."
3,Search for bank robbery suspect shuts down Hig...
4,Opened Graffiti request via android at 4134 Mi...
5,Opened Street or Sidewalk Cleaning request via...
6,Closed Graffiti request at Intersection of Bra...
7,William Rhodes's work call for Beauty @ Root D...
8,#DarkestPossibleTimeline https://t.co/AglO5D5hNx
9,Opened Street or Sidewalk Cleaning request via...


Now feed the data into classifier to filter out those tobacco tweets.

In [10]:
# running the classifier
tob_preds = clf_tob.predict(df_text)

D:\Anaconda3\envs\py36\lib\site-packages\sklearn\feature_extraction\text.py:1015: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  if hasattr(X, 'dtype') and np.issubdtype(X.dtype, np.float):


In [11]:
len(tob_preds)

87764

In [12]:
type(list(tob_preds))
tob_preds

array(['NOT-Tobacco', 'Tobacco', 'NOT-Tobacco', ..., 'NOT-Tobacco',
       'NOT-Tobacco', 'NOT-Tobacco'], dtype='<U11')

In [14]:
# finding out how many tweets there were in each group
# numpy.unique(tob_preds, return_counts=True)
print("NOT-Tobacco:", np.unique(tob_preds, return_counts=True)[1][0],
      "Tobacco:",np.unique(tob_preds, return_counts=True)[1][1])

NOT-Tobacco: 86492 Tobacco: 1272


In [15]:
indexTobacco = np.where(tob_preds == 'Tobacco')
indexTobacco = indexTobacco[0].tolist()
len(indexTobacco)
# indexTobacco

1272

In [16]:
# Double checked the index from tob_preds is correct.
# for i in indexTobacco:
#     print(tob_preds[i])

In [17]:
tob_preds[:20]

array(['NOT-Tobacco', 'Tobacco', 'NOT-Tobacco', 'NOT-Tobacco',
       'NOT-Tobacco', 'NOT-Tobacco', 'NOT-Tobacco', 'NOT-Tobacco',
       'NOT-Tobacco', 'Tobacco', 'NOT-Tobacco', 'NOT-Tobacco',
       'NOT-Tobacco', 'NOT-Tobacco', 'NOT-Tobacco', 'NOT-Tobacco',
       'NOT-Tobacco', 'NOT-Tobacco', 'NOT-Tobacco', 'NOT-Tobacco'],
      dtype='<U11')

In [18]:
dfTobacco = df.iloc[indexTobacco,]

In [19]:
dfTobacco.head(3)

,contributors,coordinates,created_at,display_text_range,entities,extended_entities,extended_tweet,favorite_count,favorited,filter_level,...,quoted_status,quoted_status_id,quoted_status_id_str,retweet_count,retweeted,source,text,timestamp_ms,truncated,user
1,NaN,"{'type': 'Point', 'coordinates': [-122.433851,...",2017-01-12 21:34:11,NaN,"{'hashtags': [], 'urls': [{'url': 'https://t.c...",NaN,NaN,0,False,low,...,NaN,NaN,NaN,0,False,"<a href=""http://spotreporters.com"" rel=""nofoll...",Opened Street or Sidewalk Cleaning request via...,2017-01-12 21:34:11.692,False,"{'id': 1589692776, 'id_str': '1589692776', 'na..."
9,NaN,"{'type': 'Point', 'coordinates': [-122.405413,...",2017-01-18 20:13:38,"[0, 140]","{'hashtags': [], 'urls': [{'url': 'https://t.c...",NaN,{'full_text': 'Opened Street or Sidewalk Clean...,0,False,low,...,NaN,NaN,NaN,0,False,"<a href=""http://spotreporters.com"" rel=""nofoll...",Opened Street or Sidewalk Cleaning request via...,2017-01-18 20:13:38.842,True,"{'id': 1589692776, 'id_str': '1589692776', 'na..."
78,NaN,"{'type': 'Point', 'coordinates': [-122.405531,...",2017-01-09 18:05:21,"[0, 140]","{'hashtags': [], 'urls': [{'url': 'https://t.c...",NaN,{'full_text': 'Opened Street or Sidewalk Clean...,0,False,low,...,NaN,NaN,NaN,0,False,"<a href=""http://spotreporters.com"" rel=""nofoll...",Opened Street or Sidewalk Cleaning request via...,2017-01-09 18:05:21.031,True,"{'id': 1589692776, 'id_str': '1589692776', 'na..."


In [124]:
# dfTobacco.text[:30]
for i in dfTobacco.text[:30]:
    print(i)

Opened Street or Sidewalk Cleaning request via android at 969 Fell St https://t.co/8H4KRcGjzt. Mattress.
Opened Street or Sidewalk Cleaning request via android at Intersection of Pine St &amp; Quincy St… https://t.co/EyXllNNLgK
Opened Street or Sidewalk Cleaning request via android at 746 Commercial St https://t.co/boz0KEC2Lt. Actual outside… https://t.co/6spgW7T2cu
Opened Street or Sidewalk Cleaning request via iphone at 465 Natoma St https://t.co/4Xax9TTefE. Broken, dumped toil… https://t.co/ibakwPqM4b
Opened Street or Sidewalk Cleaning request via iphone at Intersection of Gough St &amp; Mccoppin St… https://t.co/nnT4XgqHv9
Opened Street or Sidewalk Cleaning request via android at Intersection of Sycamore St &amp; Valencia St… https://t.co/uQTHRkozb7
Opened Street or Sidewalk Cleaning request via web at 666 Ellis St Tenderloin https://t.co/JQRwD9aRMX. Feces on the sidewalk.
Coffee and dessert. @ Nespresso Boutique &amp; Bar https://t.co/NaIAyV0VG0
Opened Street or Sidewalk Cleaning 

It's so weird that all classified tobacco related tweets have nothing to do with tobacco. Then we print all the tweets data below to see if all tweets data contains location like above.

In [125]:
for i in df.text[:30]:
    print(i)

Opened Street or Sidewalk Cleaning request via android at 3154 17th St https://t.co/nR9l9sWgkf. Piles of trash. https://t.co/rYeD6jSgnr
Opened Street or Sidewalk Cleaning request via android at 969 Fell St https://t.co/8H4KRcGjzt. Mattress.
Just posted a photo @ Cliff House, San Francisco https://t.co/uDQtPy4zjV
Search for bank robbery suspect shuts down Highway 17 in Santa Cruz Mountains https://t.co/XpiXx9LuQL
Opened Graffiti request via android at 4134 Mission St https://t.co/yyGs3jGWNO. Graffiti on bus shelter window. https://t.co/viVrR2EPpC
Opened Street or Sidewalk Cleaning request via iphone at 743 Broderick St https://t.co/CnBkw883DR. https://t.co/ZxN5Cv0dJh
Closed Graffiti request at Intersection of Brady St &amp; Colton St https://t.co/QkCOYoY4XU. Case closed. case resolved. https://t.co/RDAiwH04QN
William Rhodes's work call for Beauty @ Root Division https://t.co/mbHmkqtE1g
#DarkestPossibleTimeline https://t.co/AglO5D5hNx
Opened Street or Sidewalk Cleaning request via androi

The answer is not. Then we trying to print the features and the related coefficients to see what features matters.

In [22]:
clf_tob.steps

[('getter', ItemGetter(key='text')),
 ('tfidf',
  TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
          dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
          lowercase=True, max_df=1.0, max_features=200000, min_df=1,
          ngram_range=(1, 3), norm='l2', preprocessor=None, smooth_idf=True,
          stop_words=None, strip_accents=None, sublinear_tf=False,
          token_pattern='(?u)\\b\\w\\w+\\b',
          tokenizer=<function tokenize at 0x00000205FCCFB620>, use_idf=True,
          vocabulary=None)),
 ('clf',
  LogisticRegression(C=139.67415702201885, class_weight=None, dual=False,
            fit_intercept=True, intercept_scaling=1, max_iter=100,
            multi_class='ovr', n_jobs=-1, penalty='l2', random_state=None,
            solver='liblinear', tol=0.000449897709599141, verbose=0,
            warm_start=None))]

In [23]:
LR_tob = clf_tob.steps[2][1]

In [24]:
# coefficent = LR_tob.coef_
coefficients = LR_tob.coef_.tolist()[0]
type(coefficients)
# len(coefficent.tolist()[0])
max(coefficients)
coefficients.index(max(coefficients))
coefficients[15713]

46.848000675020025

In [25]:
# LR_tob.intercept_
# LR_tob.classes_ 
# LR_tob.n_iter_ 
# LR_tob.get_params

In [26]:
tfidf_tob = clf_tob.get_params()['tfidf']
features = tfidf_tob.get_feature_names()
print('length of features: ', len(features))
type(features)

length of features:  200000


list

In [27]:
featuresAndCoef = {'features':features,'coefficients':coefficients}

In [28]:
df_featuresAndCoef = pd.DataFrame(featuresAndCoef)
# df_featuresAndCoef.head(20)

The most important features.

In [29]:
df_featuresAndCoef.sort_values(by=['coefficients'], ascending=False).head(40)

,features,coefficients
15713,cigarettes,46.848001
177829,vaping,45.527301
15634,cigarette,42.503140
161073,tobacco,37.335308
15794,cigs,37.218617
177397,vape,36.357769
15589,cigar,34.601034
29019,hookah,33.055269
15532,cig,32.934133
142743,smoking,31.356351


The least important features.

In [30]:
df_featuresAndCoef.sort_values(by=['coefficients'], ascending=True).head(40)

,features,coefficients
16454,company,-18.917868
37352,kong,-10.662821
151530,time to smoke,-9.971389
186941,weed,-9.784976
109998,pipes,-9.671286
217,""" i don't",-9.661308
60804,my brudds,-9.609917
3330,59,-8.744774
17075,crack,-8.735898
108361,petty,-8.714689


In [31]:
df_featuresAndCoef.iloc[186941] # Double check to make sure we make the ranking above right.

features           weed
coefficients   -9.78498
Name: 186941, dtype: object

Test the bad performance is because the label within the input data or not.

In [33]:
initialData = pd.read_csv("./data/used training data/tob.csv")
initialData.head(10)
initialData = initialData[['text']]
initialData.head(10)

,text
0,gets left on read by her teacher '
1,lol give that bitch a cape cause she SUPER mad
2,the truth is not meant to hurt its meant to in...
3,bbcnews I give you this toast Here s to the da...
4,Lmaoo Don't Get Shit Twisted
5,but last year was so weird it just got to a po...
6,BBCanada Sindy to Bruno Neda defends keeping D...
7,BBCanada Sindy to Bruno Only person who doesnt...
8,BBCanada Sindy told Bruno that Ika got upset w...
9,Coca Cola doesn't make an ad by Friday I WILL ...


In [34]:
clf_tob = pickle.load(open('./tobacco classifiers/classifier1_tobacco.p', 'rb'))
result = clf_tob.predict(initialData)

D:\Anaconda3\envs\py36\lib\site-packages\sklearn\feature_extraction\text.py:1015: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  if hasattr(X, 'dtype') and np.issubdtype(X.dtype, np.float):


In [36]:
# len(result)
print("NOT-Tobacco:", np.unique(result, return_counts=True)[1][0],
      "Tobacco:",np.unique(result, return_counts=True)[1][1])

NOT-Tobacco: 22492 Tobacco: 3676


In [37]:
indexTobacco_initial = np.where(result == 'Tobacco')
indexTobacco_initial = indexTobacco_initial[0].tolist()
initialData.iloc[indexTobacco_initial,].head(10)

,text
12,dyou reckon i can give up ciggs for lent again...
19,hookah bar w it '
22,cigarette smoking kills you at 50 weed smoking...
27,1 WEEK OF NO CIGARETTES FOR
51,understand that cigarette vapour or halitosis ...
54,on the sofa dead Woke up from a coma pulled up...
58,-MENTION- -MENTION- -MENTION- Bates Cdn #vape ...
62,"-MENTION- -MENTION- I don't like ""7 yrs old"" (..."
63,"-MENTION- -MENTION- I don't like ""ironic"" (#np..."
64,"-MENTION- -MENTION- I don't like ""piano man"" (..."


Go back again to see what's the prediction.

In [46]:
textToPredict = dfTobacco.text
textToPredict = pd.DataFrame(textToPredict)
clf_tob = pickle.load(open('./tobacco classifiers/classifier1_tobacco.p', 'rb'))
result_textToPredict = clf_tob.predict(textToPredict)
# Actually all is labeled Tobacco. I didn't make the index wrong.
# for i in result_textToPredict:
#     print(i)

D:\Anaconda3\envs\py36\lib\site-packages\sklearn\feature_extraction\text.py:1015: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  if hasattr(X, 'dtype') and np.issubdtype(X.dtype, np.float):


------------------------------------------
Generate some tobacco dataset myself.

In [48]:
type(dfTobacco.text)

pandas.core.series.Series

In [73]:
generateText = ["I love cigarettes",
                "I don't love cigarettes",
                "I love vaping",
                "vaping and cigarettes",
                "I need a cigarette",
                "I'm on the street smoking a cigarette",
                "We're #hiring! Click to apply: Expanding SF Retail Firm Seeking Junior Accountant - https://t.co/7bOsjbYFdi #Accountemps #SanFrancisco, CA",
                "I'm at @Benihana in San Francisco, CA w/ @drewg",
                "Opened Street or Sidewalk Cleaning request via", # Tobacco
                "Opened Street or",
                "Sidewalk Cleaning request via",
                "Sidewalk Cleaning", # Tobacco
                "request via",
                "Sidewalk",
                "Cleaning", # Tobacco
                "Opened Street or Sidewalk request via" # Not Tobacco
                
                
                
                
               ]
# generateText = ['I love cigarettes'
                
#                ]
generateInput = {'text':generateText}
generateToPredict = pd.DataFrame(generateInput)
generateToPredict

,text
0,I love cigarettes
1,I don't love cigarettes
2,I love vaping
3,vaping and cigarettes
4,I need a cigarette
5,I'm on the street smoking a cigarette
6,We're #hiring! Click to apply: Expanding SF Re...
7,"I'm at @Benihana in San Francisco, CA w/ @drewg"
8,Opened Street or Sidewalk Cleaning request via
9,Opened Street or


In [74]:
clf_tob = pickle.load(open('./tobacco classifiers/classifier1_tobacco.p', 'rb'))
# running the classifier
result_generateToPredict = clf_tob.predict(generateToPredict)
result_generateToPredict

D:\Anaconda3\envs\py36\lib\site-packages\sklearn\feature_extraction\text.py:1015: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  if hasattr(X, 'dtype') and np.issubdtype(X.dtype, np.float):


array(['Tobacco', 'Tobacco', 'Tobacco', 'Tobacco', 'Tobacco', 'Tobacco',
       'NOT-Tobacco', 'NOT-Tobacco', 'Tobacco', 'NOT-Tobacco', 'Tobacco',
       'Tobacco', 'NOT-Tobacco', 'NOT-Tobacco', 'Tobacco', 'NOT-Tobacco'],
      dtype='<U11')

It turns out that it's all because the word "cleaning". Now check what's the coefficient of "cleaning".

In [80]:
# df_featuresAndCoef.ix[:, 'cigarettes']
df_featuresAndCoef.loc[df_featuresAndCoef['features'] ==  'cleaning']

,features,coefficients
15926,cleaning,10.31724


It's so weird. I go back to the data that Jason has used to train the classifier. In all tobacco labeled tweets, 'cleaning' only appears 4 times.

------------------------
First, check the prediction probability. I go back to the source code of function predict(). It turns out the funtion just directly take the score value calculateb by funtion decision_function(). If the score is greater than 0, then it's 'Tobacco', else it's 'Not-Tobacco'. In other words, the threshold is 0.

In [83]:
textToPredict = dfTobacco.text
textToPredict = pd.DataFrame(textToPredict)
clf_tob = pickle.load(open('./tobacco classifiers/classifier1_tobacco.p', 'rb'))
result_textToPredict = clf_tob.decision_function(textToPredict)
# len(result_textToPredict)
result_textToPredict

D:\Anaconda3\envs\py36\lib\site-packages\sklearn\feature_extraction\text.py:1015: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  if hasattr(X, 'dtype') and np.issubdtype(X.dtype, np.float):


array([0.05318358, 0.23617969, 0.42242443, ..., 0.04150259, 0.3094653 ,
       0.23736232])

In [87]:
# df_text
all_textToPredict = pd.DataFrame(df_text)
clf_tob = pickle.load(open('./tobacco classifiers/classifier1_tobacco.p', 'rb'))
result_all_textToPredict = clf_tob.decision_function(all_textToPredict)
# len(result_all_textToPredict)
result_all_textToPredict

D:\Anaconda3\envs\py36\lib\site-packages\sklearn\feature_extraction\text.py:1015: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  if hasattr(X, 'dtype') and np.issubdtype(X.dtype, np.float):


array([-0.28596195,  0.05318358, -6.97263249, ..., -5.64834359,
       -5.36897124, -4.90257439])

In [88]:
sum(result_all_textToPredict>0)

1272

In [89]:
# Change prediction method to funtion predict_proba()
# df_text
all_textToPredict = pd.DataFrame(df_text)
clf_tob = pickle.load(open('./tobacco classifiers/classifier1_tobacco.p', 'rb'))
result_all_textToPredictProba = clf_tob.predict_proba(all_textToPredict)
# len(result_all_textToPredictProba)
result_all_textToPredictProba

D:\Anaconda3\envs\py36\lib\site-packages\sklearn\feature_extraction\text.py:1015: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  if hasattr(X, 'dtype') and np.issubdtype(X.dtype, np.float):


array([[5.71007266e-01, 4.28992734e-01],
       [4.86707239e-01, 5.13292761e-01],
       [9.99063695e-01, 9.36305041e-04],
       ...,
       [9.96489022e-01, 3.51097765e-03],
       [9.95362683e-01, 4.63731684e-03],
       [9.92627323e-01, 7.37267721e-03]])

In [90]:
len(result_all_textToPredictProba)

87764

In [99]:
tmpSum = 0
for i in result_all_textToPredictProba:
#     print(i)
    tmpSum += i.argmax()

In [100]:
tmpSum

1272

So for function predict_proba(), it just extract the index of bigger value as the predicted class label.

-----------------------------------
Second, remove all the word 'cleaning' in the data and re-predict.

In [123]:
df_text_filtered = df_text.replace(to_replace ="Cleaning", value ="", regex=True)
df_text_filtered.head(20)

,text
0,Opened Street or Sidewalk request via android...
1,Opened Street or Sidewalk request via android...
2,"Just posted a photo @ Cliff House, San Francis..."
3,Search for bank robbery suspect shuts down Hig...
4,Opened Graffiti request via android at 4134 Mi...
5,Opened Street or Sidewalk request via iphone ...
6,Closed Graffiti request at Intersection of Bra...
7,William Rhodes's work call for Beauty @ Root D...
8,#DarkestPossibleTimeline https://t.co/AglO5D5hNx
9,Opened Street or Sidewalk request via android...


Now we trying to input the data after filtered into classifier.

In [126]:
# running the classifier
tob_preds_filtered = clf_tob.predict(df_text_filtered)

D:\Anaconda3\envs\py36\lib\site-packages\sklearn\feature_extraction\text.py:1015: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  if hasattr(X, 'dtype') and np.issubdtype(X.dtype, np.float):


In [127]:
# finding out how many tweets there were in each group
# numpy.unique(tob_preds, return_counts=True)
print("NOT-Tobacco:", np.unique(tob_preds_filtered, return_counts=True)[1][0],
      "Tobacco:",np.unique(tob_preds_filtered, return_counts=True)[1][1])

NOT-Tobacco: 87665 Tobacco: 99


In [128]:
indexTobacco_filtered = np.where(tob_preds_filtered == 'Tobacco')
indexTobacco_filtered = indexTobacco_filtered[0].tolist()
len(indexTobacco_filtered)
# indexTobacco

99

In [138]:
dfTobacco_filtered = df_text_filtered.iloc[indexTobacco_filtered,]
dfTobacco_filtered.head(3)

,text
279,Coffee and dessert. @ Nespresso Boutique &amp;...
1538,"#ChoppinGame! New year, adding new methods! @ ..."
2186,"I'm at 724 Hookah in San Francisco, CA https:/..."


In [139]:
for i in dfTobacco_filtered.text:
    print(i)

Coffee and dessert. @ Nespresso Boutique &amp; Bar https://t.co/NaIAyV0VG0
#ChoppinGame! New year, adding new methods! @ Cigar Bar &amp; Grill https://t.co/fNE5L5rxF2
I'm at 724 Hookah in San Francisco, CA https://t.co/Fj8DSv8hNi
Don't buy Nestle products 🎨 #explorecalifornia #sanfrancisco #cliffhouse #california #graffiti @… https://t.co/JAvM9D2h5L
Interesting flavor, smokey and crisp - Drinking a Smokes Mcgotes by @blackhammerbrew @ Black Hammer Brewing  — https://t.co/mCRblrl0IM
Fresh from Cuba. Thanks @air_andrea for the gift, smoking this on my… https://t.co/T2PhdzDs2b
I'm at Cigar Bar &amp; Grill - @850cigarbar in San Francisco, CA https://t.co/tOFw3TyqXW
I'm off to the airport in about 12 hours. Didn't do much today except pack, clean and relax one… https://t.co/TfTVXXzH73
Drinking a Smokes Mcgotes by @blackhammerbrew @ Black Hammer Brewing — https://t.co/K4T4UADepQ
Headed to Bar Fluxus #SF #downtown #Bowie #singalong @kittenonthekeys 6-9 1/6/17 #cocktails @… https://t.co/BvkW3H

In [142]:
df_featuresAndCoef.loc[df_featuresAndCoef['features'] ==  'drink']

,features,coefficients
19848,drinking,6.953225


In [147]:
df_featuresAndCoef.loc[df_featuresAndCoef['features'] ==  'coffee']

,features,coefficients
16084,coffee,6.809538


In [141]:
df_featuresAndCoef.loc[df_featuresAndCoef['features'] ==  'bar']

,features,coefficients
10778,bar,10.041871


In [148]:
df_featuresAndCoef.loc[df_featuresAndCoef['features'] ==  'wine']

,features,coefficients
188911,wine,4.515863
